# Web search with Bing

In [ ]:
import os
import openai
from IPython.display import display, HTML, Markdown
from pprint import pprint

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.callbacks import OpenAICallbackHandler

totals_cb = OpenAICallbackHandler()

print(totals_cb)

### Set-up Bing Search API account

Go to https://www.microsoft.com/en-us/bing/apis/bing-web-search-api, press Try Now and follow instructions... Select Free Tier for education and proof of concept use cases.

Get API Key (Key 1) and put it into an Environment Variable, possibly in `~/.env` file.

Check the [Bing Search API Pricing](https://www.microsoft.com/en-us/bing/apis/pricing)

In [ ]:
bing_subscription_key = os.environ['BING_SUBSCRIPTION_KEY']

In [ ]:
from langchain.utilities import BingSearchAPIWrapper

bing_search = BingSearchAPIWrapper(
    bing_subscription_key = bing_subscription_key,
    bing_search_url = "https://api.bing.microsoft.com/v7.0/search",
)

In [ ]:
bing_search.run("LangChain BingSearchAPIWrapper")

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="chat_history")],
}

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = ChatOpenAI(model="gpt-4", temperature=0.0) # agent requires chat LLM
        
tools = [
    Tool(
        name="Search",
        func=bing_search.run,
        description="useful to search the web to find answers",
    )
]

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    memory=memory,
    agent_kwargs=agent_kwargs,
    verbose=True
)

In [ ]:
response = agent.run("Tell me about LangChain BingSearchAPIWrapper", callbacks=[totals_cb])

display(Markdown(response))

In [ ]:
response = agent.run("How to set up Bing Search API to enable use of LangChain BingSearchAPIWrapper?", callbacks=[totals_cb])

display(Markdown(response))

In [ ]:
response = agent.run("Develop a plan of actions using web search and execute it to answer following question: What other famous people where born in the hometown of the 10th person walking on the Moon?", callbacks=[totals_cb])

display(Markdown(response))

In [ ]:
print(totals_cb)